In [4]:
# imports
import pandas as pd

In [5]:
# import custom libraries and scripts
# sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(".")
sys.path.append("..")

from defaults import *
from extract_dataframe import read_json
from extract_dataframe import TweetDfExtractor
from clean_tweets_dataframe import Clean_Tweets

In [7]:
# read csv
tweets_df = pd.read_csv("../data/clean_data.csv")
tweets_df

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,status_count,retweet_count,screen_name,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,2022-08-07 22:31:20+00:00,Twitter for Android,RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,en,4,8097,2,i_ameztoy,i_ameztoy,20497,2621,unknown,"[{'text': 'City', 'indices': [132, 137]}]","[{'screen_name': 'i_ameztoy', 'name': 'Iban Am...",unknown
1,2022-08-07 22:31:16+00:00,Twitter for Android,RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,en,691,5831,201,ZIisq,ZIisq,65,272,unknown,"[{'text': 'China', 'indices': [18, 24]}, {'tex...","[{'screen_name': 'IndoPac_Info', 'name': 'Indo...",unknown
2,2022-08-07 22:31:07+00:00,Twitter for Android,"China even cut off communication, they don't a...",0.000000e+00,0.000000,en,0,1627,0,Fin21Free,Fin21Free,85,392,unknown,"[{'text': 'XiJinping', 'indices': [127, 137]}]","[{'screen_name': 'ZelenskyyUa', 'name': 'Волод...",Netherlands
3,2022-08-07 22:31:06+00:00,Twitter for Android,"Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,en,0,1627,0,Fin21Free,Fin21Free,85,392,unknown,"[{'text': 'XiJinping', 'indices': [9, 19]}]",[],Netherlands
4,2022-08-07 22:31:04+00:00,Twitter for iPhone,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,en,1521,18958,381,VizziniDolores,VizziniDolores,910,2608,unknown,[],"[{'screen_name': 'ChinaUncensored', 'name': 'C...","Ayent, Schweiz"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7435,2022-08-06 18:04:28+00:00,Twitter Web App,RT @metesohtaoglu: 📌📸 Map of #China's possible...,0.000000e+00,1.000000,en,9,209,3,PelosiLibArmy,PelosiLibArmy,8,128,unknown,"[{'text': 'China', 'indices': [29, 35]}, {'tex...","[{'screen_name': 'metesohtaoglu', 'name': 'Met...","Seattle, WA"
7436,2022-08-06 18:03:58+00:00,Twitter for iPhone,RT @NEVERBOW: China is doing #exactly what #Ru...,2.500000e-01,0.250000,en,1,15235,1,SonnyMullins13,SonnyMullins13,224,240,unknown,"[{'text': 'exactly', 'indices': [29, 37]}, {'t...","[{'screen_name': 'NEVERBOW', 'name': 'P K', 'i...",unknown
7437,2022-08-06 18:03:48+00:00,Twitter for iPhone,Minister Wu is crystal clear in his @BBCNews i...,1.583333e-01,0.419444,en,0,1791,0,TECO_Toronto,TECO_Toronto,955,202,0.0,"[{'text': 'Taiwan', 'indices': [168, 175]}, {'...","[{'screen_name': 'BBCNews', 'name': 'BBC News ...","Toronto, Canada"
7438,2022-08-06 18:03:46+00:00,Twitter for Android,Reports say that #China is planning to seize #...,0.000000e+00,0.000000,en,0,268,0,samserjio93,samserjio93,22,59,unknown,"[{'text': 'China', 'indices': [17, 23]}, {'tex...",[],unknown
